In [1]:
from qiskit_aer import AerSimulator
from qiskit import *
import supermarq , os
from qiskit.providers.fake_provider import *
import numpy as np
emulator_aer_dm = Aer.get_backend('aer_simulator_density_matrix')

In [2]:
all_fakes=[FakeAlmadenV2,FakeArmonkV2,FakeAthensV2,FakeAuckland,FakeBelemV2,FakeBoeblingenV2,FakeBogotaV2,FakeBrooklynV2,FakeBurlingtonV2,FakeCairoV2,FakeCambridgeV2,FakeCasablancaV2,FakeEssexV2,FakeGeneva,FakeGuadalupeV2,FakeHanoiV2,FakeJakartaV2,FakeJohannesburgV2,FakeKolkataV2,FakeLagosV2,FakeLimaV2,FakeLondonV2,FakeManhattanV2,FakeManilaV2,FakeMelbourneV2,FakeMontrealV2,FakeMumbaiV2,FakeNairobiV2,FakeOslo,FakeOurenseV2,FakeParisV2,FakePerth,FakePrague,FakePoughkeepsieV2,FakeQuitoV2,FakeRochesterV2,FakeRomeV2,FakeSantiagoV2,FakeSherbrooke,FakeSingaporeV2,FakeSydneyV2,FakeTorontoV2,FakeValenciaV2,FakeVigoV2,FakeWashingtonV2,FakeYorktownV2]
import random; selected_fakes = random.sample(all_fakes, 5)
selected_fakes

[qiskit.providers.fake_provider.backends.manila.fake_manila.FakeManilaV2,
 qiskit.providers.fake_provider.backends.ourense.fake_ourense.FakeOurenseV2,
 qiskit.providers.fake_provider.backends.guadalupe.fake_guadalupe.FakeGuadalupeV2,
 qiskit.providers.fake_provider.backends.athens.fake_athens.FakeAthensV2,
 qiskit.providers.fake_provider.backends.burlington.fake_burlington.FakeBurlingtonV2]

In [3]:
benchmarks = [
    (supermarq.bit_code.BitCode(3, 3, [1, 0, 1]), "bitcode3"),
    (supermarq.ghz.GHZ(3), "ghz3"),
    (supermarq.hamiltonian_simulation.HamiltonianSimulation(3), "hsim3"),
    (supermarq.mermin_bell.MerminBell(3), "marmin_bell_3"),
    (supermarq.qaoa_fermionic_swap_proxy.QAOAFermionicSwapProxy(3), "qaoa_swap_proxy3"),
    (supermarq.qaoa_vanilla_proxy.QAOAVanillaProxy(3), "qaoa_vanilla_proxy3"),]  

for i in benchmarks:
    circuit=i[0].qiskit_circuit()
    print("\n------------------------------",i[1],"-----------------------------")
    
    #print(circuit.draw())


------------------------------ bitcode3 -----------------------------

------------------------------ ghz3 -----------------------------

------------------------------ hsim3 -----------------------------

------------------------------ marmin_bell_3 -----------------------------

------------------------------ qaoa_swap_proxy3 -----------------------------

------------------------------ qaoa_vanilla_proxy3 -----------------------------


In [4]:
benchmark_features = {}
for benchmark, label in benchmarks:
    benchmark_features[label] = [
        supermarq.converters.compute_communication_with_qiskit(benchmark.qiskit_circuit()),
        supermarq.converters.compute_depth_with_qiskit(benchmark.qiskit_circuit()),
        supermarq.converters.compute_entanglement_with_qiskit(benchmark.qiskit_circuit()),
        supermarq.converters.compute_liveness_with_qiskit(benchmark.qiskit_circuit()),
        supermarq.converters.compute_measurement_with_qiskit(benchmark.qiskit_circuit()),
        supermarq.converters.compute_parallelism_with_qiskit(benchmark.qiskit_circuit()),]
print(benchmark_features)

{'bitcode3': [0.4, 0.5, 0.8571428571428571, 0.6142857142857143, 0.46153846153846156, 0.0], 'ghz3': [0.6666666666666666, 1.0, 0.6666666666666666, 0.6666666666666666, 0.0, 0], 'hsim3': [0.6666666666666666, 1.0, 0.26666666666666666, 0.7333333333333333, 0.0, 0.25], 'marmin_bell_3': [1.0, 1.0, 0.4375, 0.6666666666666666, 0.0, 0.11538461538461542], 'qaoa_swap_proxy3': [0.6666666666666666, 1.0, 0.5, 0.6666666666666666, 0.0, 0.09999999999999998], 'qaoa_vanilla_proxy3': [1.0, 1.0, 0.4, 0.6666666666666666, 0.0, 0.125]}


In [ ]:
jobs =[]
for benchmark, label in benchmarks:
    circuit=benchmark.qiskit_circuit()
    for backend in selected_fakes:
        fake=backend
        job = fake().run(circuit, shots=1000)
        result=job.result()
        jobs.append((label,benchmark,fake,job, job.result()))
        #counts=job.result(0).count()
jobs

In [37]:
jobs =[]
for backend in selected_fakes:
    fake=backend
    for benchmark, label in benchmarks:
        circuit=benchmark.qiskit_circuit()
        job = fake().run(circuit, shots=1000)
        result=job.result()
        jobs.append((label,benchmark,fake,job, job.result()))
        #counts=job.result(0).count()
#jobs

[('bitcode3',
  qiskit.providers.fake_provider.backends.manila.fake_manila.FakeManilaV2,
  Result(backend_name='aer_simulator', backend_version='0.13.0', qobj_id='', job_id='b18c56b4-c25c-4a04-a66e-badb9bc07a4c', success=True, results=[ExperimentResult(shots=1000, success=True, meas_level=2, data=ExperimentResultData(counts={'0x6f': 1, '0x42e': 1, '0x6e': 1, '0x55f': 1, '0x4df': 1, '0x40f': 1, '0x57': 1, '0x55': 1, '0x457': 3, '0x579': 1, '0x475': 1, '0x42b': 4, '0x67b': 1, '0x57b': 1, '0x57d': 1, '0x43d': 3, '0x45d': 1, '0x42f': 6, '0x47b': 25, '0x5f': 3, '0x436': 1, '0x7b': 2, '0x4bf': 3, '0x7f': 26, '0x4ef': 1, '0x45f': 26, '0x67e': 2, '0x47a': 1, '0x53f': 2, '0x47d': 24, '0x46f': 27, '0x47f': 557, '0x53b': 1, '0x43f': 149, '0x47e': 16, '0x67f': 10, '0x3f': 9, '0x434': 1, '0x46b': 1, '0x66d': 1, '0x4ff': 5, '0x57f': 7, '0x43e': 10, '0x46d': 1, '0x477': 26, '0x3b': 1, '0x17': 2, '0x467': 1, '0x1e': 1, '0x437': 7, '0x4bb': 1, '0x46a': 1, '0x43b': 7, '0x41f': 6, '0x52f': 1, '0x53': 1, 

In [89]:
def cut(j):
    l= str(i)
    import re; parts = l.split(".")
    j=parts[5]
    #print(j)
    return j


device = []    


for i in selected_fakes:
    for label,circuit,backend,job,result in jobs:
        if backend == i:
            counts = job.result(0).get_counts()
            score = circuit.score(counts)
            print(circuit)
            print(" BACKEND = ",backend, "; \n CIRCUIT NAME = ",label,"; \n SCORE = ",score)
            #supermarq.plotting.plot_results([score], [label])
            print(label)
            #device_scores[label] = score
            #print(device_scores,"device_scores[label]==",device_scores[label],"; score == ",score )
            #i=+1
print(device_scores)

 BACKEND =  <class 'qiskit.providers.fake_provider.backends.manila.fake_manila.FakeManilaV2'> ; 
 CIRCUIT NAME =  bitcode3 ; 
 SCORE =  0.0
bitcode3
 BACKEND =  <class 'qiskit.providers.fake_provider.backends.manila.fake_manila.FakeManilaV2'> ; 
 CIRCUIT NAME =  ghz3 ; 
 SCORE =  0.811550865276332
ghz3
 BACKEND =  <class 'qiskit.providers.fake_provider.backends.manila.fake_manila.FakeManilaV2'> ; 
 CIRCUIT NAME =  hsim3 ; 
 SCORE =  0.9031132760733926
hsim3
 BACKEND =  <class 'qiskit.providers.fake_provider.backends.manila.fake_manila.FakeManilaV2'> ; 
 CIRCUIT NAME =  marmin_bell_3 ; 
 SCORE =  0.4965
marmin_bell_3
 BACKEND =  <class 'qiskit.providers.fake_provider.backends.manila.fake_manila.FakeManilaV2'> ; 
 CIRCUIT NAME =  qaoa_swap_proxy3 ; 
 SCORE =  0.39866615780321835
qaoa_swap_proxy3
 BACKEND =  <class 'qiskit.providers.fake_provider.backends.manila.fake_manila.FakeManilaV2'> ; 
 CIRCUIT NAME =  qaoa_vanilla_proxy3 ; 
 SCORE =  0.8440000341262182
qaoa_vanilla_proxy3
 BACKEND 

In [ ]:
def create_global_variable(name, value):
    globals()[name] = value

    device_scores = 'device_score_' +parts[5]

def cut(j):
    l= str(i)
    import re; parts = l.split(".")
    j=parts[5]
    #print(j)
    return j


j=cut(i)
            device='device_score_'
            device_scores = 'device_score_' + str(j)
            print(device_scores)
            x = {}
            x = device_scores
            device_scores = x
            globals()[device_scores] = {}

            print(type(device_scores))
            print("==================")


In [59]:
i=1
names=[]
       
#l= str(backend)
#import re; parts = l.split(".")
#names=parts[5]
#print(names)


for label,circuit,backend,job,result in jobs: 
    device='device_score_'
    device_scores = 'device_score_' +parts[5]
    print(device_scores)
    device_scores = {}  
    print(device_scores)

    #print(i)
    #counts = job.result(0).get_counts()
    #score = circuit.score(counts)
    #print(circuit)
    #print(" BACKEND = ",backend, "; \n CIRCUIT NAME = ",label,"; \n SCORE = ",score)
    #supermarq.plotting.plot_results([score], [label])
    #print(label)
    #device_scores[label] = score
    #print(device_scores,"device_scores[label]==",device_scores[label],"; score == ",score )
    i+=1
print(device_scores)

device_score_fake_burlington
{}
device_score_fake_burlington
{}
device_score_fake_burlington
{}
device_score_fake_burlington
{}
device_score_fake_burlington
{}
device_score_fake_burlington
{}
device_score_fake_burlington
{}
device_score_fake_burlington
{}
device_score_fake_burlington
{}
device_score_fake_burlington
{}
device_score_fake_burlington
{}
device_score_fake_burlington
{}
device_score_fake_burlington
{}
device_score_fake_burlington
{}
device_score_fake_burlington
{}
device_score_fake_burlington
{}
device_score_fake_burlington
{}
device_score_fake_burlington
{}
device_score_fake_burlington
{}
device_score_fake_burlington
{}
device_score_fake_burlington
{}
device_score_fake_burlington
{}
device_score_fake_burlington
{}
device_score_fake_burlington
{}
device_score_fake_burlington
{}
device_score_fake_burlington
{}
device_score_fake_burlington
{}
device_score_fake_burlington
{}
device_score_fake_burlington
{}
device_score_fake_burlington
{}
{}


In [28]:


for i in range(len(selected_fakes)):
    #device='device_score_'
    device_scores = 'device_score_' +str(i)
    device_scores = {}   
    for benchmark, label in benchmarks:
        #print(label)
        
        #i=i+1
        #print(i)
        for label,benchmark,backend,job,result in jobs:
            counts = job.result(0).get_counts()
            score = benchmark.score(counts)
            #print(circuit)
            print(" BACKEND = ",backend, "; \n CIRCUIT NAME = ",label,"; \n SCORE = ",score)
            #supermarq.plotting.plot_results([score], [label])
            #print("===========================================================")
            device_scores[label] = score
            #print(device_scores,"device_scores[label]==",device_scores[label],"; score == ",score )
print(device_scores)


 BACKEND =  <class 'qiskit.providers.fake_provider.backends.manila.fake_manila.FakeManilaV2'> ; 
 CIRCUIT NAME =  bitcode3 ; 
 SCORE =  0.0
 BACKEND =  <class 'qiskit.providers.fake_provider.backends.ourense.fake_ourense.FakeOurenseV2'> ; 
 CIRCUIT NAME =  bitcode3 ; 
 SCORE =  0.0
 BACKEND =  <class 'qiskit.providers.fake_provider.backends.guadalupe.fake_guadalupe.FakeGuadalupeV2'> ; 
 CIRCUIT NAME =  bitcode3 ; 
 SCORE =  0.0
 BACKEND =  <class 'qiskit.providers.fake_provider.backends.athens.fake_athens.FakeAthensV2'> ; 
 CIRCUIT NAME =  bitcode3 ; 
 SCORE =  0.0
 BACKEND =  <class 'qiskit.providers.fake_provider.backends.burlington.fake_burlington.FakeBurlingtonV2'> ; 
 CIRCUIT NAME =  bitcode3 ; 
 SCORE =  0.0
 BACKEND =  <class 'qiskit.providers.fake_provider.backends.manila.fake_manila.FakeManilaV2'> ; 
 CIRCUIT NAME =  ghz3 ; 
 SCORE =  0.791877196100256
 BACKEND =  <class 'qiskit.providers.fake_provider.backends.ourense.fake_ourense.FakeOurenseV2'> ; 
 CIRCUIT NAME =  ghz3 ; 
 

In [ ]:
print(device_scores_0)

In [ ]:
for label,circuit,backend,job,result in jobs:
    print(label,"counts :", counts, "; score =",score)
    supermarq.plotting.plot_results([score], [label])
    print("-----------------------------------------------------------")

In [ ]:
device_scores = {}
for label, job, benchmark in jobs:
    counts = job.result(0).get_counts()
    print(counts)
    score = benchmark.score(counts)
    print(score)
    device_scores[label] = score

In [ ]:
supermarq.plotting.plot_correlations(benchmark_features,
                                     [device_scores_fake, device_scores],
                                     ["PC", "CD", "Ent", "Liv", "Mea", "Par"],
                                     device_name=["aer_dm_sim","FakeRueschlikon"],
                                     show=True,)

In [ ]:
for benchmark, label in benchmarks:
    supermarq.plotting.plot_benchmark([label, [label], [benchmark_features[label]]],spoke_labels=["PC", "CD", "Ent", "Liv", "Mea", "Par"],)
